# Implemention of the behavior Pattern Method
nach 
Diamantini, C., Genga, L. & Potena, D. Behavioral process mining for unstructured processes. J Intell Inf Syst 47, 5–32 (2016). https://doi.org/10.1007/s10844-016-0394-7

In [1]:
import pm4py
from pm4py.objects.log.obj import EventLog
from pm4py.objects.log.obj import Event
from pm4py.objects.log.obj import Trace
import os

## Import XES File
from https://pm4py.fit.fraunhofer.de/documentation#item-impoort-xes

Event Log, und Trace: https://pm4py.fit.fraunhofer.de/static/assets/api/2.2.16/pm4py.objects.log.html?highlight=objects%20log%20obj%20eventlog#pm4py.objects.log.obj.EventLog

EventLog[] => Trace

EventLog[][] => Event of Trace

In [2]:
from pm4py.objects.log.importer.xes import importer as xes_importer

In [3]:
path = "datasets/02_TestCompletedFFF_onlyTaskItems_simpleFilter.xes"
# path = "datasets/01_TestCompletedFFF_IDtoString_removeTransitionClassifier.xes"
log = xes_importer.apply(path)

parsing log, completed traces :: 100%|██████████| 81/81 [00:00<00:00, 86.04it/s]


## Get Casual Relation with Inductive Miner infrequent
from https://pm4py.fit.fraunhofer.de/documentation#item-3-2 

Dauer ca. 2min

In [4]:
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.objects.petri_net.exporter import exporter as pnml_exporter
from pm4py.objects.petri_net.importer import importer as pnml_importer


In [5]:
variantIM = inductive_miner.Variants.IMf
# Log
net, initial_marking, final_marking = inductive_miner.apply(log,variant=variantIM)
pnml_exporter.apply(net, initial_marking, "output/petri_net_full.pnml", final_marking=final_marking) 


""" # Log gefiltert
filtered_log = pm4py.filter_case_size(log, 40, 200) # 40, 200 => 540;
net, initial_marking, final_marking = inductive_miner.apply(filtered_log,variant=variantIM)
pnml_exporter.apply(net, initial_marking, "output/petri_filtered.pnml", final_marking=final_marking) 
"""

' # Log gefiltert\nfiltered_log = pm4py.filter_case_size(log, 40, 200) # 40, 200 => 540;\nnet, initial_marking, final_marking = inductive_miner.apply(filtered_log,variant=variantIM)\npnml_exporter.apply(net, initial_marking, "output/petri_filtered.pnml", final_marking=final_marking) \n'

### Import Petri Net

In [6]:
# Full Petri Net
# net, initial_marking, final_marking = pnml_importer.apply(os.path.join("output/petri_net_full.pnml"))

# Filtered Petri Net
# net, initial_marking, final_marking = pnml_importer.apply(os.path.join("output/petri_filtered.pnml"))


## From Petri Netz zu Instance Ordering  
### Casual Relation:

Zwei events i, j haben eine Causal Relation, wenn i erst abgeschlossen werden muss, damit j ausgeführt werden kann.

### Instance Ordering: 

i, j und k sind Events; i<k<j
Es besteht eine CR Relation zwischen i und j;
und
(Es darf keine CR Relation zwischen i und k:
Oder 
Es darf keine CR Relation zwischen k und j geben;)

Bezug zum PetriNetz, welches als Grundlage für die CR Relation verwendet wird:

Ein Event ist im Petri Netz eine Transition; der Aktivitäten Name kann über Label aufgerufen werden.

Siehe hierzu auch
https://pm4py.fit.fraunhofer.de/static/assets/api/2.2.16/pm4py.objects.petri_net.html?highlight=petrinet#pm4py.objects.petri_net.obj.PetriNet

https://pm4py.fit.fraunhofer.de/documentation#item-4-2

In [7]:
from pm4py.objects.petri_net.obj import PetriNet


In [17]:
# Print Einsicht
## print(len(log)) 
## type is trace, is also eine sequence
event = log[2][0]
print(event)
print(event['concept:name'])

""" place = net.places
print(place)
print(place.name)

arc = place.in_arcs[0]
print(arc.source.name, arc.source.label) """

""" places = net.places
for place in places:
 print("\nPLACE: "+place.name)
 for arc in place.in_arcs:
  print(arc.source.name, arc.source.label) """

""" print(net.arcs)
print(net.places)
print(net.transitions) """
""" print(net.places)
print(net.transitions) """

""" for trans in net.transitions:
    print(trans.in_arcs) """

## print(net.arcs[0].name)


{'lifecycle:transition': 'start', 'time:timestamp': datetime.datetime(2019, 11, 21, 8, 50, 53, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'cdb_process_id': 'P00062660', 'concept:instance': 327087, 'concept:name': 'Product Modification Responsible is correctly assigned?', 'description': 'The task has been completed.  parts to be changed added', 'division': 'TD', 'msgtype': 'DONE', 'org:resource': '1079aa99-2699-11ec-8124-0050568b0d8f', 'task_id': 'T00010248'}
Product Modification Responsible is correctly assigned?


' for trans in net.transitions:\n    print(trans.in_arcs) '

In [9]:
def calculateCaRelToPreTransitions(trans: PetriNet.Transition):
    casualRelTrans = set()
    for transArc in trans.in_arcs:
    ## get inarcs of place source
        for placeArc in transArc.source.in_arcs:
            casualRelTrans.add(placeArc.source)
    return casualRelTrans

def genInstanceOrdering(trace: Trace, net: PetriNet):
    traceList = list(trace) ## Aktivitäten könne doppelt vorkommen, aber muss nur einmal Cr berechnen, daher set, Problme set erkennt gleiche strings nicht als unterchiedlich!
    casualRelation = set()
    for trans in net.transitions:
        currentTransLabel = trans.label
        def isTransInTrace(event):
            return currentTransLabel == event['concept:name'] 
        filteredItems = set(filter(isTransInTrace, traceList))
        ## Achtung eine Aktivität kann mehrfach auftreten, wie umgehen, damit?
        ##print(len(filteredItems))
        if(len(filteredItems)>0):
            ## print(currentTransLabel, "is in Trace") 
            casualRelTransitions = calculateCaRelToPreTransitions(trans)
            ## remove for performance; TODO: überprüfe Annahme: jede Aktivität taucht nur einmal als Transition auf!
            for event in filteredItems:
                traceList.remove(event)
            for casualRelTrans in casualRelTransitions:
                casualRelation.add((casualRelTrans.label, currentTransLabel))
    #  Rückgabe ist menge der 
    return casualRelation


Verbesserung des Algorithmus beim Iteration über alle Logs: Verhindere doppelte CR Berechnung, wenn schon für Aktivität Berechnet

Ansatz:
Sichere berechnetet CR Relationen und speichere Aktivität für die berechnet in Liste. Frage in jeder Iteration ab, ob schon berechnet?

Frage Wirklich Verbesserung? Da nun jedes mal über die Liste der Aktivitäten, für die schon berechnet wurde Iteriert wird.

In [14]:

""" for trace in log:
    crSet = genCR(trace,net)
    print(len(crSet))
    for cr in crSet: 
        print(cr) """

<class 'pm4py.objects.log.obj.EventLog'>


' for trace in log:\n    crSet = genCR(trace,net)\n    print(len(crSet))\n    for cr in crSet: \n        print(cr) '

In [11]:
## alter Ansatz

def getTuplesWithEvent(ev: Event, arcs):
    arcsList = set()
    #print(arcs)
    for arc in arcs:
        #print(type(arc))
        """ if(arc == ev['concept:name'] or arc== ev['concept:name']):
            print('YAY')
            arcsList.add(arc) """
    return arcsList
    
def checkCasualRelation(ev1: Event, ev2:Event, net: PetriNet)-> bool:
    tuple = (ev1['concept:name'], ev2['concept:name'])
    ## print(tuple)
    ## TODO: wie machen wir hier die Vergleich: greife auf die CasualRelation Eigenschaft zu
    arcList = getTuplesWithEvent(ev1, net.arcs)
    arcList2 = getTuplesWithEvent(ev1, arcList)
    hasRelation = len(arcList2)>0
    ## hasRelation = tuple in net.arcs
    if(hasRelation):
        print('YAY Treffer')
        print(tuple)
    return hasRelation

from collections.abc import Sequence
def getInstanceOrdering(trace: Trace, net: PetriNet)->Sequence[tuple]:
    instanceOrderingSet = set()
    for idx, event in enumerate(trace):
        nextIndex = idx+1
        ##print(trace)
        if(nextIndex > len(trace)-1):
            print("For break")
            break
        event2 = trace[nextIndex]
        hasCasualRelation = False

        while(not hasCasualRelation):
            if(nextIndex > len(trace)-1):
                print("while break")
                break
            event2 = trace[nextIndex]
            hasCasualRelation = checkCasualRelation(event, event2, net)
            nextIndex = nextIndex+1

        if (hasCasualRelation):
            instanceOrderingSet.add((event, event2))

    return instanceOrderingSet

## Instance Graphen

### Event Log
https://pm4py.fit.fraunhofer.de/static/assets/api/2.2.16/pm4py.objects.log.html?highlight=eventlog#pm4py.objects.log.obj.EventLog



In [18]:

class InstanceGraph:
    vertices = {}

    edges = {}

    def __init__(self, trace: set, instaceOrdering: Sequence[tuple]):
        self.vertices = trace
        self.edges = instaceOrdering

    def getName(self, label):
        return label

instanceGraphSet = set()
for trace in log:
    nodes = set()
    nodeEventDict = dict()
    # Nodes
    for event in trace:
        vertice = event['division'] + ':::' + event['concept:name']
        nodes.add(vertice)
        nodeEventDict[vertice] = event
    # Kanten, aber nicht in Edge Form
    instOrdering = genInstanceOrdering(trace, net)
    edges = instOrdering

    instanceGraphSet.append((nodes, edges, nodeEventDict))


AttributeError: 'set' object has no attribute 'append'

### Visualisierung Instance Graphen


Bibliothek NetworkX Data Struktur

https://networkx.org/documentation/stable/tutorial.html

https://networkx.org/documentation/stable/reference/classes/digraph.html

https://www.youtube.com/watch?v=xREnpVUbkFI

## Substructure Evaluation

Kapitel 5.2.1 Substructures evaluation

diversity,
completeness (coverage of the model),
Repräsentativität,
Häufigkeit (substrukturen),